In [96]:
import yahooquery as yq
import numpy as np
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd

In [97]:
# Get current list of Spy from wiki
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data = pd.read_html(url)

In [98]:
sp500 = data[0].iloc[:,[0,1,6]]
sp500.columns = ['ticker', 'name', 'date']

In [99]:
sp500

,ticker,name,date
0,MMM,3M,1976-08-09
1,AOS,A. O. Smith,2017-07-26
2,ABT,Abbott,1964-03-31
3,ABBV,AbbVie,2012-12-31
4,ABMD,Abiomed,2018-05-31
...,...,...,...
499,YUM,Yum! Brands,1997-10-06
500,ZBRA,Zebra,2019-12-23
501,ZBH,Zimmer Biomet,2001-08-07
502,ZION,Zions Bancorp,2001-06-22


In [100]:
# Get rows where date is missing or not formatted correctly.
mask = sp500['date'].str.strip().str.fullmatch('\d{4}-\d{2}-\d{2}')
mask.loc[mask.isnull()] = False
mask = mask == False
sp500[mask].head()

,ticker,name,date
29,AMD,AMD,NaN
51,T,AT&T,1983-11-30 (1957-03-04)
126,ED,Con Edison,NaN
131,GLW,Corning,NaN
140,DHR,Danaher,NaN


In [101]:
current = sp500.copy()
current.loc[mask, 'date'] = '1900-01-01'
current.loc[:, 'date'] = pd.to_datetime(current['date'])

In [102]:
# Fill in missing data
adjustments = data[1]
adjustments.columns =['date', 'ticker_added','name_added', 'ticker_removed', 'name_removed', 'reason']
adjustments.head()

,date,ticker_added,name_added,ticker_removed,name_removed,reason
0,"April 4, 2022",CPT,Camden,PBCT,People's United Financial,S&P 500 constituent M&T Bank Corp. acquired Pe...
1,"March 2, 2022",MOH,Molina Healthcare,INFO,IHS Markit,S&P 500 constituent S&P Global Inc. acquired I...
2,"February 15, 2022",NDSN,Nordson,XLNX,Xilinx,S&P 500 constituent Advanced Micro Devices acq...
3,"February 3, 2022",NaN,NaN,GPS,Gap,Market capitalization change.[9]
4,"February 2, 2022",CEG,Constellation Energy,NaN,NaN,S&P 500 and 100 constituent Exelon Corp. spun ...


In [103]:
# create additons df
additions = adjustments[~adjustments['ticker_added'].isnull()][['date','ticker_added', 'name_added']]
additions.columns = ['date','ticker', 'name']
additions['action'] = 'added'

In [104]:
#create removals df
removals = adjustments[~adjustments['ticker_removed'].isnull()][['date','ticker_removed', 'name_removed']]
removals.columns = ['date','ticker','name']
removals['action'] = 'removed'

In [105]:
#Merge df
historical = pd.concat([additions,removals])
historical.head()

,date,ticker,name,action
0,"April 4, 2022",CPT,Camden,added
1,"March 2, 2022",MOH,Molina Healthcare,added
2,"February 15, 2022",NDSN,Nordson,added
4,"February 2, 2022",CEG,Constellation Energy,added
5,"December 20, 2021",SBNY,Signature Bank,added


In [106]:
missing = current[~current['ticker'].isin(historical['ticker'])].copy()
missing['action'] = 'added'
missing = missing[['date','ticker','name','action']]
missing.head()

,date,ticker,name,action
0,1976-08-09,MMM,3M,added
2,1964-03-31,ABT,Abbott,added
7,1981-07-29,ADM,ADM,added
8,1997-05-05,ADBE,Adobe,added
9,1981-03-31,ADP,ADP,added


In [107]:
sp500_history = pd.concat([historical, missing])
sp500_history = sp500_history.sort_values(by=['date','ticker'], ascending=[False, True])
sp500_history = sp500_history.drop_duplicates(subset=['date','ticker'])
sp500_history

,date,ticker,name,action
121,"September 8, 2016",CHTR,Charter Communications,added
121,"September 8, 2016",EMC,EMC Corporation,removed
122,"September 6, 2016",MTD,Mettler Toledo,added
122,"September 6, 2016",TYC,Tyco International,removed
217,"September 5, 2012",LYB,LyondellBasell,added
...,...,...,...,...
485,1900-01-01 00:00:00,WAT,Waters,added
493,1900-01-01 00:00:00,WHR,Whirlpool,added
484,1900-01-01 00:00:00,WM,Waste Management,added
491,1900-01-01 00:00:00,WRK,WestRock,added


In [108]:
sp500_history['date'] = pd.to_datetime(sp500_history['date'])

In [109]:
sp500_history.sort_values(by='date', axis=0)
sp500_history

,date,ticker,name,action
121,2016-09-08,CHTR,Charter Communications,added
121,2016-09-08,EMC,EMC Corporation,removed
122,2016-09-06,MTD,Mettler Toledo,added
122,2016-09-06,TYC,Tyco International,removed
217,2012-09-05,LYB,LyondellBasell,added
...,...,...,...,...
485,1900-01-01,WAT,Waters,added
493,1900-01-01,WHR,Whirlpool,added
484,1900-01-01,WM,Waste Management,added
491,1900-01-01,WRK,WestRock,added


In [110]:
mask = (sp500_history['date']> '2000-01-01') & (sp500_history['date'] <='2021-12-31')
universe = sp500_history.loc[mask]

In [111]:
universe.sort_values(by='date', axis=0)

,date,ticker,name,action
13,2000-06-05,A,Agilent Technologies,added
294,2000-06-07,SMS,Shared Medical Systems,removed
294,2000-06-07,SBUX,Starbucks,added
293,2000-07-27,JDSU,JDS Uniphase,added
293,2000-07-27,RAD,RiteAid,removed
...,...,...,...,...
7,2021-12-20,WU,Western Union,removed
5,2021-12-20,LEG,Leggett & Platt,removed
6,2021-12-20,HBI,Hanesbrands,removed
5,2021-12-20,SBNY,Signature Bank,added


In [112]:
universe.to_csv('S&P500 from 2000-2021')